In [2]:
!pip install -q youtube-transcript-api langchain langchain-community \
               faiss-cpu tiktoken python-dotenv

from pathlib import Path
from dotenv import load_dotenv
import os, importlib

env_path = Path(".env")
if not env_path.exists():
    env_path.write_text("# Add your OpenRouter key here:\n# OPENROUTER_API_KEY=sk-...\n")
    print(".env created — open it and paste your OpenRouter key (don’t share it here).")

# Load the environment variables
load_dotenv()

# Confirm key presence safely
key_present = "FOUND ✅" if os.getenv("OPENROUTER_API_KEY") else "NOT FOUND ❌"
print(f"OPENROUTER_API_KEY: {key_present}")

# Quick import check
def show_version(pkg_name, import_name=None):
    try:
        m = importlib.import_module(import_name or pkg_name)
        v = getattr(m, "__version__", "unknown")
        print(f"{pkg_name:20s} version: {v}")
    except Exception as e:
        print(f"{pkg_name:20s} import failed: {e}")

show_version("youtube_transcript_api")
show_version("langchain", "langchain")
show_version("langchain-community", "langchain_community")
show_version("faiss", "faiss")
show_version("tiktoken", "tiktoken")
show_version("python-dotenv", "dotenv")

OPENROUTER_API_KEY: FOUND ✅
youtube_transcript_api version: unknown
langchain            version: 0.3.27
langchain-community  version: 0.3.31
faiss                version: 1.12.0
tiktoken             version: 0.12.0
python-dotenv        version: unknown



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install -q --upgrade youtube-transcript-api==0.6.2

import youtube_transcript_api
print("Available methods in YouTubeTranscriptApi:", dir(YouTubeTranscriptApi))

Available methods in YouTubeTranscriptApi: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'fetch', 'list']



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptFound
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
video_id = "Gfr50f6ZBvo"

try:
    api = YouTubeTranscriptApi()
    transcripts = api.list(video_id)

    transcript_obj = None
    for t in transcripts:
        if "en" in t.language_code:
            transcript_obj = t
            break
    if transcript_obj is None:
        transcript_obj = transcripts[0]  # fallback

    transcript_snippets = transcript_obj.fetch()
    transcript = " ".join(snippet.text for snippet in transcript_snippets)

    print("✅ Transcript fetched successfully!")
    print(f"Transcript length: {len(transcript)} characters")

except TranscriptsDisabled:
    print("❌ Captions are disabled for this video.")
    transcript = ""
except NoTranscriptFound:
    print("❌ No transcript found for this video.")
    transcript = ""
except Exception as e:
    print(f"⚠️ Unexpected error: {e}")
    transcript = ""

if transcript:
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.create_documents([transcript])
    print(f"✅ Split into {len(chunks)} chunks.")
else:
    chunks = []

✅ Transcript fetched successfully!
Transcript length: 133836 characters
✅ Split into 168 chunks.


In [14]:
!pip install -q openai langchain-openai


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import os

embedding_model = "text-embedding-3-small"

try:
    embeddings = OpenAIEmbeddings(
        model=embedding_model,
        openai_api_base="https://openrouter.ai/api/v1",
        openai_api_key=os.getenv("OPENROUTER_API_KEY")
    )

    # --- Create FAISS vector store ---
    vectorstore = FAISS.from_documents(chunks, embeddings)
    vectorstore.save_local("faiss_youtube_index")

    print("✅ Embeddings created and FAISS index saved as 'faiss_youtube_index'")
    print(f"Vector count: {len(chunks)}")

except Exception as e:
    print(f"⚠️ Error while creating embeddings: {e}")


✅ Embeddings created and FAISS index saved as 'faiss_youtube_index'
Vector count: 168


In [29]:
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
import os

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.getenv("OPENROUTER_API_KEY")
)

# 🔹 Reload FAISS with consistent embedding dimension
vectorstore = FAISS.load_local("faiss_youtube_index", embeddings, allow_dangerous_deserialization=True)

# 🔹 Retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",  # 👈 Enable Max Marginal Relevance
    search_kwargs={
        "k": 6,           # Number of chunks to return
        "fetch_k": 15,    # Candidate chunks to consider
        "lambda_mult": 0.5  # Balance between relevance (0) and diversity (1)
    }
)

# 🔹 LLM: LLaMA 3.3 (OpenRouter)
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-8b-instruct:free",
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.getenv("OPENROUTER_API_KEY"),
    temperature=0.3
)

prompt = PromptTemplate(
    template="""
    You are a helpful assistant.
    Use ONLY the context from the transcript below to answer the question.
    If the context is insufficient, say you don't know.

    Context:
    {context}

    Question:
    {question}
    """,
    input_variables=['context', 'question']
)

In [31]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# --- Create conversational memory ---
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# --- Create a conversational retrieval chain ---
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    verbose=True
)

C:\Users\vaibh\AppData\Local\Temp\ipykernel_3236\623883206.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [40]:
def format_docs(retrieved_docs):
    return "\n\n".join(doc.page_content for doc in retrieved_docs)

parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

parser = StrOutputParser()
main_chain = parallel_chain | prompt | llm | parser

query = "In this video, does nuclear fusion topics discussed?"
docs = retriever.get_relevant_documents(query)

# --- Generate the answer ---
response = qa_chain.invoke({"question": query})

# Depending on the return type, extract the answer safely
if isinstance(response, dict) and "answer" in response:
    answer = response["answer"]
else:
    answer = response

print("✅ Retrieved", len(docs), "docs for reflection.")
print("🧠 Chatbot Response:\n", answer)

C:\Users\vaibh\AppData\Local\Temp\ipykernel_3236\3988924115.py:13: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)




> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: In this video, does nuclear fusion topics discussed?
Assistant: Yes, nuclear fusion is a topic discussed in this video. The speaker mentions collaborating with EPFL in Switzerland on a test reactor, and also talks about their work on "magnetic control of tokamak plasmas to deep reinforcement learning" to solve nuclear fusion using deep RL. They also mention that they are looking to tackle the next problem in the fusion area and are in talks with fusion startups.
Human: In this video, does nuclear fusion topics discussed?
Assistant: Yes, nuclear fusion is mentioned in the video as one of the areas where AI can potentially help, specifically in the context of overcoming the challenges of building massive fusion reactors and containing plasma.
Human: In this video

In [41]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# --- Reflection prompt ---
reflection_prompt = PromptTemplate.from_template("""
You are a critical scientific reviewer.
Review the following answer in light of the retrieved context.

Context:
{context}

Answer:
{answer}

Check for:
1. Factual accuracy — does the answer align with the context?
2. Completeness — does it cover key points from the context?
3. Confidence — if unsure, explain why.

Now rate confidence (High/Medium/Low) and suggest improvements if needed.
""")

reflection_chain = LLMChain(
    llm=llm,  # same LLaMA model you used before
    prompt=reflection_prompt,
    verbose=True
)


In [42]:
reflection = reflection_chain.run({
    "context": docs,
    "answer": answer
})

print("🤔 Reflection:\n", reflection)

C:\Users\vaibh\AppData\Local\Temp\ipykernel_3236\3239076956.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  reflection = reflection_chain.run({




> Entering new LLMChain chain...
Prompt after formatting:

You are a critical scientific reviewer.
Review the following answer in light of the retrieved context.

Context:
[Document(id='b8d0a4c3-4647-4bb9-b426-74b2cf508340', metadata={}, page_content="in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion 

In [43]:
if "Low" in reflection or "Medium" in reflection:
    improved_answer = llm.predict(
        f"Improve the following answer using the feedback:\n\nAnswer:\n{answer}\n\nFeedback:\n{reflection}"
    )
    print("✅ Improved Answer:\n", improved_answer)
else:
    improved_answer = answer

C:\Users\vaibh\AppData\Local\Temp\ipykernel_3236\702870945.py:2: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  improved_answer = llm.predict(


✅ Improved Answer:
 Here is an improved answer based on the feedback:

Yes, nuclear fusion is a topic discussed in the video. The speaker mentions collaborating with EPFL in Switzerland on a test reactor for nuclear fusion, and also discusses their own work on using AI to tackle problems in nuclear fusion. Specifically, they highlight the use of deep reinforcement learning to control high-temperature plasmas in tokamak devices, which is a crucial step in achieving sustainable nuclear fusion. The speaker also emphasizes the potential of AI to solve the challenges of nuclear fusion, including controlling plasma stability and confinement. Notably, their work has shown promising results, with the ability to hold plasmas in specific shapes for record amounts of time. Overall, the speaker's research on nuclear fusion using AI is a promising area of study with significant potential applications in the field of energy production, offering a potentially game-changing solution to the world's gro